### Importing relevant libraries

In [1]:
import dash
# import dash_core_components as dcc
from dash import dcc
from dash import html
# import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State

### Loading titanic dataset

In [2]:
df = pd.read_csv('titanic.csv', na_values='?')

### Preprocessing our data

In [3]:
# define columns as category instead of numerical
df = df.astype({"pclass":'category', "survived":'category'})

# create a column with yes and no for survived instead of 0 and 1 
df['survived_str'] = df['survived'].replace({0: 'No', 1:'Yes'})

# categorize age
group_labels = ['young_elderly', 'able', 'young_elderly']
bins = [0,15,50,100]
df['age_cat'] = pd.cut(df['age'], bins=bins, labels=group_labels, ordered=False)

# categorize fare 
group_labels = ['poor', 'normal', 'rich']
bins = [-1,9,75,1000]
df['fare_cat'] = pd.cut(df['fare'], bins=bins, labels=group_labels)

# categorize cabin column as either exist or no  
df['cabin_cat'] = df['cabin'].fillna('no_cabin')
df.loc[df['cabin_cat']!='no_cabin', 'cabin_cat'] = 'cabin'

# categorize on sex and class

# define the function that will do the segregation 
def segregate(x):
    if x['sex']=='female' and x['pclass']==1 and x['fare_cat']=='rich' :
        return 'rich women'
    elif x['sex']=='male' and x['pclass']==3 and x['fare_cat']=='poor':
        return 'poor men'
    else :
        return "neither"    
    
df['segregated'] = df.apply(segregate, axis=1)

### Defining plotting functions

In [4]:
def get_survivor_class(pclass=None):
    
    # % of survivors among each pclass
    if pclass==None:
        return px.histogram(df, x="survived_str", color="pclass", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"}, 
             title="Percentage of survivors among each Pclass",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    else:
        return px.histogram(df[df['pclass']==pclass], x="survived_str", color="pclass", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"}, 
             title="Percentage of survivors among each Pclass",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    
fig1=get_survivor_class()    

In [5]:
def get_survivor_sex(sex=None):
    
# % of survivors among each sex

    if sex==None:
        return px.histogram(df, x="survived_str", color="sex", barmode="group", histfunc='count', histnorm='percent',
             labels={"survived_str": "Survived", "percent": "%"}, 
             title="Percentage of survivors among each Sex",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")

    else:
        return px.histogram(df[df['sex']==sex], x="survived_str", color="sex", barmode="group", histfunc='count', histnorm='percent',
             labels={"survived_str": "Survived", "percent": "%"}, 
             title="Percentage of survivors among each Sex",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    
fig2= get_survivor_sex()

In [6]:
def get_survivor_age(age_cat=None):
    # % of survivors among each Age categroy
    if age_cat==None:
        return px.histogram(df[df['age_cat'].notnull()], x="survived_str", color="age_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each Age categroy",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    
    else:
        return px.histogram(df[(df['age_cat']==age_cat) & (df['age_cat'].notnull())], x="survived_str", color="age_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each Age categroy",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
fig3= get_survivor_age()

In [7]:
def get_survivor_fare(fare_cat=None):
    # % of survivors among each Fare price categroy

    if fare_cat==None:
        return px.histogram(df[df['fare'].notnull()], x="survived_str", color="fare_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each price categroy",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    
    else:
        return px.histogram(df[(df['fare'].notnull()) & (df['fare_cat']==fare_cat)], x="survived_str", color="fare_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each Fare price categroy",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
fig4= get_survivor_fare()

In [8]:
def get_survivor_cabin(cabin_cat=None):
# % of survivors among existing Cabin passengers 

    if cabin_cat==None:
        return px.histogram(df, x="survived_str", color="cabin_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each Cabin state",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    else:
        return px.histogram(df[df['cabin_cat']==cabin_cat], x="survived_str", color="cabin_cat", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title="Percentage of survivors among each Cabin state",
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
fig5= get_survivor_cabin()

In [9]:
def get_survivor_layer(layer=None):
# % of survivors among each layer

    if layer==None:
        return px.histogram(df, x="survived_str", color="segregated", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title='Percentage of survivors among each "Sex & Class" categroy', 
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
    else:
        return px.histogram(df[df['segregated']==layer], x="survived_str", color="segregated", barmode="group", histfunc='count', histnorm='percent', 
             labels={"survived_str": "Survived", "percent": "%"},
             title='Percentage of survivors among each "Sex & Class" categroy', 
             color_discrete_sequence=px.colors.qualitative.Pastel).update_layout(yaxis_title="%")
fig6= get_survivor_layer()

### css link

In [10]:
my_css_link = 'https://codepen.io/chriddyp/pen/bWLwgP.css'


### app.py

In [ ]:
app=dash.Dash(external_stylesheets=[my_css_link])

app.layout= html.Div([
    
    
    html.H1('Titanic Analysis',style={'textAlign':'center','color':'Blue'}),
    html.Br(),
    html.Hr(),
    html.P('All Rights Reserved',style={'background':'black','color':'white'}),

    
    
    html.Div([
        
        html.Div([
#         html.H1('Scatter Plot'),
         html.P([
                "Built with ",
                html.A("Dash", href="https://plot.ly/products/dash/"),
                 "."
            ], style={"textAlign": "center","color": "Gray"}),
        
        
#          html.P([
#                 "Work done by ",
#                 html.A("Atef El-Hennawy", href="https://www.facebook.com/5oBo2"),
#                 "& ",
#                 html.A("Ahmed Mady", href="https://www.facebook.com/ahmedmady1994"),
#                  "."
#             ], style={"text-align": "right"}),
            
        dcc.Graph(id='graph1',figure=fig1),
        dcc.Dropdown(id='dropdown1',
                    options=[{'label':str(pclass),'value':(pclass)} for pclass in df['pclass'].unique()],
                    value=None,
                    placeholder='choose a pclass'),
    
        
        dcc.Graph(id='graph2',figure=fig2),
        dcc.Dropdown(id='dropdown2',
                    options=[{'label':str(sex),'value':str(sex)} for sex in df['sex'].unique()],
                    value=None,
                    placeholder='choose sex')
                   ],className='four columns'),
        
    ]),
    
        html.Div([
        
#         html.H1('Scatter Plot'),
             html.P([
                "Built with ",
                html.A("Plotly", href="https://plotly.com/"),
                 "."
            ], style={"textAlign": "center","color": "Gray"}),
            
        dcc.Graph(id='graph3',figure=fig3),
        dcc.Dropdown(id='dropdown3',
                    options=[{'label':str(age_cat),'value':str(age_cat)} for age_cat in df['age_cat'].unique()],
                    value=None,
                    placeholder='choose a age_cat'),
            
        
        dcc.Graph(id='graph4',figure=fig4),
        dcc.Dropdown(id='dropdown4',
                    options=[{'label':str(fare_cat),'value':str(fare_cat)} for fare_cat in df['fare_cat'].unique()],
                    value=None,
                    placeholder='choose a fare_cat')       
                    ],className='four columns'),
    
     
        html.Div([
        
#         html.H1('Scatter Plot'),
          html.P([
                "Work done by ",
                html.A("Atef El-Hennawy", href="https://www.facebook.com/5oBo2"),
                "& ",
                html.A("Ahmed Mady", href="https://www.facebook.com/ahmedmady1994"),
                 "."
            ], style={"text-align": "right"}),
            
        dcc.Graph(id='graph5',figure=fig5),
        dcc.Dropdown(id='dropdown5',
                    options=[{'label':str(cabin_cat),'value':str(cabin_cat)} for cabin_cat in df['cabin_cat'].unique()],
                    value=None,
                    placeholder='choose a cabin_cat'),
            
        
        dcc.Graph(id='graph6',figure=fig6),
        dcc.Dropdown(id='dropdown6',
                    options=[{'label':str(layer),'value':str(layer)} for layer in df['segregated'].unique()],
                    value=None,
                    placeholder='choose a layer')
                    ],className='four columns'),
     
    ])
        
@app.callback(

        Output(component_id='graph1',component_property='figure'),
        Output(component_id='graph2',component_property='figure'),
        Output(component_id='graph3',component_property='figure'),
        Output(component_id='graph4',component_property='figure'),
        Output(component_id='graph5',component_property='figure'),
        Output(component_id='graph6',component_property='figure'),


        Input(component_id='dropdown1',component_property='value'),
        Input(component_id='dropdown2',component_property='value'),
        Input(component_id='dropdown3',component_property='value'),
        Input(component_id='dropdown4',component_property='value'),
        Input(component_id='dropdown5',component_property='value'),
        Input(component_id='dropdown6',component_property='value'),

)

def update_my_year(pclass,sex,age_cat,fare_cat,cabin_cat,layer):
    
    return get_survivor_class(pclass=pclass),\
           get_survivor_sex(sex=sex),\
           get_survivor_age(age_cat),\
           get_survivor_fare(fare_cat),\
           get_survivor_cabin(cabin_cat),\
           get_survivor_layer(layer)         
               
          
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2021 10:05:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:48] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:48] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:48] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:05:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:10:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:10:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:14:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 10:17:13] "POST /_dash-update-component HTTP/1.